In [ ]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests
import re

In [ ]:
STATE_2_STATE_URL = "https://www.census.gov/data/tables/time-series/demo/geographic-mobility/state-to-state-migration.html"

In [ ]:
# convert href urls to full urls
def modify_url(url):
    if url.startswith("//"):
        return "https:" + url
    else:
        return url


# returns dictionary with years as keys and file names as values
def get_file_by_year(file_urls):
    years = {}
    # Regular expression pattern to extract years (YYYY)
    pattern = r"\d{4}"
    for file_url in file_urls:
        filename = file_url.split("/")[-1]  
        # Use regex to find all matches
        matches = re.findall(pattern, filename)
        # If there are matches, save the filename with the year as key
        if matches:
            if len(matches) == 2:
                years[f"{matches[0]}_{matches[1]}"] = filename
            else:
                years[f"{matches[0]}"] = filename
        if not matches:
            continue

    return years

# extracts urls with .xls extension from the given url
def get_xls_file_urls(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    state_2_state_file_urls = []
    # get <a> tags with ".xls" extension links
    a_tags = soup.find_all('a', href=re.compile('.xls'))
    # get url for spreadsheet download
    for a_tag in a_tags:
        state_2_state_file_urls.append(modify_url(a_tag['href']))
    
    return state_2_state_file_urls


# Download files from list of urls
def download_files(urls):
    for file_url in urls:
        # Get the filename from the URL
        filename = file_url.split("/")[-1]  
        # Check if file exists
        if os.path.isfile(filename):
            print(f"File '{filename}' already exists.")
            continue
        
        # download the file
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"File '{filename}' downloaded successfully.")
        else:
            print(f"Failed to download file from URL: {file_url}")

In [ ]:
spreadsheet_urls = get_xls_file_urls(STATE_2_STATE_URL)
spreadsheets = get_file_by_year(spreadsheet_urls)

In [ ]:
for spreadsheet in spreadsheet_urls:
    print(spreadsheet)
print("="*80)
for spreadsheet, file in spreadsheets.items():
    print(spreadsheet, file)

In [ ]:
download_files(spreadsheet_urls)